# ML3. Метрики качества классификации

После того, как мы обучили классификатор, необходимо **оценить его качество**. Для этого у нас есть несколько метрик.

Самая простая и понятная метрика — *Accuracy*. По сути, это просто доля объектов, которые алгоритм отнес к верному классу.

Введем следующие обозначения.

> Например, если мы хотим обучить алгоритм распознавать людей, у которых может быть обнаружено заболевание, то больные будут попадать в **позитивный** класс, а здоровые — в **негативный**.

Тогда верно определенных больных мы обозначим за $TP$ *(true positive)*, ошибочно определенных больных — $FP$ за *(false positive)*, верно определенных здоровых — за $TN$ *(true negative)*, ошибочно определенных здоровых — $FN$ за *(false negative)*.

То есть:
- $TP$ — истинно-положительные решения;
- $TN$ — истинно-отрицательные решения;
- $FP$ — ложно-положительные решения;
- $FN$ — ложно-отрицательные решения.

Для наглядности представим все варианты решений в табличном виде:

|              | Positives(1) | Negatives(0) |
|--------------|--------------|--------------|
| Positives(1) | TP           | FP           |
| Negatives(0) | FN           | TN           |

Обратите внимание на обозначенные на картинке выше новые термины: *Type 1 error* и *Type 2 error*. Это ошибки первого и второго рода. Ошибка первого рода – это ситуация, когда отвергнута правильная нулевая гипотеза.

**Нулевая гипотеза** – предположение о том, что между двумя наблюдаемыми феноменами не существует связи. Например, мы понимаем, что обычный мужчина не может забеременеть – это является верной нулевой гипотезой. Однако принимая ложноположительное решение о том, что мужчина беременный, мы совершаем ошибку первого рода. А утверждая, что беременная женщина не беременна, мы совершаем ошибку второго рода, принимая неправильную нулевую гипотезу.

Вернемся к метрике *Accuracy*. Она вычисляется следующим образом:

$ Accuracy = \frac{TN+TP}{TP+TN+FP+FN} $

Важно отметить, что у этой метрики есть довольно существенный недостаток. Она присваивает всем объектам **одинаковый вес**. Поэтому в случае несбалансированности классов она может давать нерелевантный результат.

Рассмотрим на конкретном примере. Например, мы, мы хотим оценить работу анализатора негативных отзывов на нашем сайте. У нас есть 100 положительных отзывов, 80 из которых наш классификатор определил верно *(True Negative = 80, False Positive = 20)*, и 10 негативных, 5 из которых классификатор также определил верно *(True Positive = 5, False Negative = 5)*.

В этом случае мы будем иметь *Accuracy*:

$ Accuracy = \frac{80+5}{5+80+20+5} = 0.77 $

Однако если мы будем предсказывать все отзывы как положительные, даже те, которые являются отрицательными *(True Negative = 100, False Negative = 10)*, *Accuracy* будет очень высокой.

$ Accuracy = \frac{100+0}{0+100+0+10} = 0.9 $

При этом модель, по факту, не может определять нужные нам негативные отзывы. Для решения этой проблемы нужно перейти с общей метрики к отдельным показателям качества: *Precision, Recall, F-мера*.

## Точность и Полнота

**Точность классификации** — это доля объектов, действительно принадлежащих данному классу относительно всех объектов, которые алгоритм отнес к этому классу.

*Precision (точность)* вычисляется следующим образом:

|              | Positives(1) | Negatives(0) |
|--------------|--------------|--------------|
| Positives(1) | TP           | FP           |
| Negatives(0) |              |              |

$ Precision = \frac{TP}{TP+FP} $

**Полнота** — это доля найденных классификатором объектов, принадлежащих классу относительно всех объектов, которые принадлежат этому классу.

*Recall (полнота)* вычисляется следующим образом:

|              | Positives(1) | Negatives(0) |
|--------------|--------------|--------------|
| Positives(1) | TP           |              |
| Negatives(0) | FN           |              |

$ Recall = \frac{TP}{TP+FN} $

> Например, мы пытаемся найти больных в группе людей, которые пришли на диспансеризацию. Тогда точность классификации — это доля действительно больных среди всех, кого на диспансеризации определили как больных. А полнота классификации — это доля найденных больных среди всех больных.

**F-мера** 

Чтобы найти оптимальное соотношение этих показателей, существует метрика, которая объединяет в себе сразу и точность, и полноту *F-мера*. *F-мера* является средним гармоническим между точностью и полнотой и вычисляется по следующей формуле:

$ Fмера= \frac{2 \cdot Precision \cdot Recall }{Precision + Recall} $

## Реализация в Python

Теперь попробуем реализовать классификацию и вычислить разобранные метрики.
Для начала подгружаем библиотеки:

In [1]:
from sklearn.model_selection import train_test_split # функция, чтобы разбить данные на трейн и тест
from sklearn.linear_model import LogisticRegression # наша модель для классификации

Воспользуемся встроенным датасетом, который содержит информацию об опухолях груди:

In [9]:
from sklearn.datasets import load_breast_cancer # подгружаем датасет
breast_cancer = load_breast_cancer()

Теперь зададим зависимую и независимые переменные:

In [10]:
Y = breast_cancer.target ## Наша целевая переменная, 0 — если рака нет, 1 — если есть 
X = breast_cancer.data # X - признаки, по которым мы будем предсказывать рак 

Разбиваем выборку на обучающую и тестовую и обучаем нашу модель:

In [11]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.3)
model = LogisticRegression()
model.fit(X_train, Y_train)

/data/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Готово! Теперь осталось только вычислить необходимые метрики:

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Y_predicted = model.predict(X_val)
print(accuracy_score(Y_val,Y_predicted))
print(precision_score(Y_val,Y_predicted))
print(recall_score(Y_val,Y_predicted))
print(f1_score(Y_val,Y_predicted))

0.9298245614035088
0.9473684210526315
0.9473684210526315
0.9473684210526315


## Задание 2.1

Вы создали классификатор, который разделяет экономические и политические новости на два разных *Telegram*-канала, и теперь хотите проверить качество классификатора. За день вышло 15 политических новостей и 20 экономических.
Ваш алгоритм из 15 политических новостей отметил 9 как экономические, а из 20 экономических — 6 как политические.
Найдите метрику *Accuracy*.

Ответ округлите до сотых.

In [13]:
def accuracy(tp, tn, fp, fn):
    return (tn + tp) / (tp + tn + fp + fn)

In [ ]:
a = accuracy()

print(f'{a:.2f}')

## Задание 2.2

Загрузите встроенный в библиотеку *sklearn* датасет про ирисы с помощью функции `load_iris`. Обучите модель логистической регрессии *(random_state=50, размер тестовой выборки 0.3)* и укажите полученное значение метрики *Accucary*.

Ответ округлите до сотых.